In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import bioframe 
import pandas as pd
import numpy as np 
import os
import json
from io import StringIO
import random

import pysam
import h5py

from Bio import motifs
from Bio import pairwise2
from Bio.Seq import Seq

In [2]:
import sys

sys.path.insert(0, "/home1/smaruj/akita_utils/")

# from akita_utils import *
import akita_utils

2022-07-12 10:06:39.714455: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /spack/apps/linux-centos7-x86_64/gcc-8.3.0/python-3.9.2-uvcroioc4witkp6qf7mbebof5ix4wlb6/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/pmix-3.1.3-3sm6emyqaxapunh7rwbjvtaqoqe2e5z3/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openmpi-4.0.2-ipm3dnvlbtxawpi4ifz7jma6jgr7mexq/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openblas-0.3.8-2no6mfziiclwxb7lstxoos335gnhjpes/lib:/spack/apps/gcc/8.3.0/lib64::/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2022-07-12 10:06:39.714490: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
genome_open = pysam.Fastafile("/project/fudenber_735/genomes/mm10/mm10.fa")

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
print("Tensorflow: ", tf.__version__)

from basenji import dataset, seqnn, dna_io, stream

Tensorflow:  2.9.1


In [5]:
# NOTE
# head_i = 0 #human
# head_i = 1 #mouse
#

head_i = 1 #mouse
# head_i = 0
model_num = 1 #which fold to use


#base_dir = '/project/fudenber_735/backup/DNN_HiC/human-mouse_5-16-21/'
#model_dir = base_dir+"/f"+str(model_num)+"_c0/train/"

base_dir = "/project/fudenber_735/tensorflow_models/akita/v2/models/"
model_dir = base_dir + "/f" + str(model_num) + "c0/train/"
model_file  = model_dir + "/model" + str(head_i) + "_best.h5"


# model_dir = '/home1/fudenber/repositories/basenji/manuscripts/akita/'
# model_file = model_dir+'/model_best.h5'

params_file = model_dir + "/params.json"
# params_file -> json (dict) with model's parameters
with open(params_file) as params_open:
    params = json.load(params_open)
    params_model = params["model"]
    params_train = params["train"]
seq_length = params_model["seq_length"]
params_model["verbose"] = False

seqnn_model = seqnn.SeqNN(params_model)
print("built")

seqnn_model.restore(model_file, head_i=head_i)    # model with the mouse head
print("restored")

2022-07-12 10:06:42.149619: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /spack/apps/linux-centos7-x86_64/gcc-8.3.0/python-3.9.2-uvcroioc4witkp6qf7mbebof5ix4wlb6/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/pmix-3.1.3-3sm6emyqaxapunh7rwbjvtaqoqe2e5z3/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openmpi-4.0.2-ipm3dnvlbtxawpi4ifz7jma6jgr7mexq/lib:/spack/apps/linux-centos7-x86_64/gcc-8.3.0/openblas-0.3.8-2no6mfziiclwxb7lstxoos335gnhjpes/lib:/spack/apps/gcc/8.3.0/lib64::/home1/smaruj/software/GSL/lib:/home1/smaruj/software/HTSLIB/lib
2022-07-12 10:06:42.149662: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-12 10:06:42.149706: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (d11-05.hpc.usc.edu): /pro

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence (InputLayer)          [(None, 1310720, 4)  0           []                               
                                ]                                                                 
                                                                                                  
 stochastic_reverse_complement   ((None, 1310720, 4)  0          ['sequence[0][0]']               
 (StochasticReverseComplement)  , ())                                                             
                                                                                                  
 stochastic_shift (StochasticSh  (None, 1310720, 4)  0           ['stochastic_reverse_complement[0
 ift)                                                            ][0]']                     

In [6]:
hic_diags = params_model["diagonal_offset"]
try:
    target_crop = params_model["trunk"][-2]["cropping"]
except:
    target_crop = params_model["target_crop"]

print("hic_diags: ", hic_diags) 
print("target_crop: ", target_crop)
print("seq_length: ", seq_length)

target_length_cropped = int((seq_length//2048 - target_crop*2 - hic_diags) * ((seq_length//2048 - target_crop*2 - hic_diags) +1)/2) 
target_map_size = seq_length//2048  - target_crop*2 
triu_tup = np.triu_indices(target_map_size, 2)    # Return the indices for the upper-triangle of an (n, m) array, here k=2 (diagonal offset)
# target_map_size, target_length_cropped, triu_tup[0].shape

print("target_length_cropped: ", target_length_cropped)
print("target_map_size: ", target_map_size)
print("shape of triu_tup[0]: ", triu_tup[0].shape)

hic_diags:  2
target_crop:  64
seq_length:  1310720
target_length_cropped:  130305
target_map_size:  512
shape of triu_tup[0]:  (130305,)


In [7]:
background_file = base_dir + '../analysis/background_seqs.fa'
background_seqs = []
with open(background_file,'r') as f:
  for line in f.readlines():
    if '>' in line: continue
    background_seqs.append(dna_io.dna_1hot(line.strip())) 

In [8]:
len(background_seqs)

10

In [9]:
seq_coords_df = akita_utils.prepare_insertion_tsv(
    h5_dirs = '/project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model*/scd.h5',
    score_key = 'SCD',
    pad_flank = 0, #how much flanking sequence around the sites to include
    weak_thresh_pct = 1, # don't use sites weaker than this, might be artifacts
    weak_num = 20 ,
    strong_thresh_pct = 99, # don't use sites weaker than this, might be artifacts
    strong_num = 20 ,
    save_tsv=None, # optional filename to save a tsv
)

29 duplicates removed for  /project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model7/scd.h5
29 duplicates removed for  /project/fudenber_735/tensorflow_models/akita/v2/analysis/permute_boundaries_motifs_ctcf_mm10_model1/scd.h5
annotating each site with boundary-wide scores
filtering sites by overlap with rmsk
df prepared


In [10]:
# padding = 20

In [11]:
# num_inserts = 2
# one_side_radius = 100 
# spacer = one_side_radius - padding

In [12]:
# motif_len = 19
# multi_insert_length = num_inserts * (motif_len + 2 * one_side_radius)
# print(multi_insert_length//num_inserts)

In [13]:

def multiple_padding(seq_coords_df, 
                     background_seqs, 
                     pad_list,
                     orientation="same",
                     plotting=True, 
                     num_inserts=2, 
                     one_side_radius=100, 
                     motif_len=19):
    
    orientation_types = ["same", "convergent", "divergent"]
    if orientation not in orientation_types:
        raise ValueError("Invalid orientation. Expected one of: %s" % orientation_types)
    
    nr_seq = len(seq_coords_df)
    nr_bg = len(background_seqs)
    
    multi_insert_length = num_inserts * (motif_len + 2 * one_side_radius)
    
    
    for padding in pad_list:
        
        print(f"padding = {padding} || Started working on padding = {padding}")
        
        spacer = one_side_radius - padding
        
        offsets = []

        for i in range(num_inserts):
            offsets.append( seq_length//2 - multi_insert_length//2 + i * (multi_insert_length//2) + spacer)

        all_inserts = []

        # inserting only to the first background sequence
        
        if orientation == "same":
            for background_seq in background_seqs:
                for i in seq_coords_df.index:
                    seq_1hot = background_seq.copy()
                    this_row = seq_coords_df.iloc[i]
                    seq_1hot_CTCF_pad = dna_io.dna_1hot(genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding).upper())
                    if this_row.strand == '-': 
                        seq_1hot_CTCF_pad = dna_io.hot1_rc(seq_1hot_CTCF_pad)
                    # print(this_row.strand, Seq(genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding).upper()))
                    for offset in offsets:
                        seq_1hot[offset : offset + len(seq_1hot_CTCF_pad)] = seq_1hot_CTCF_pad
                    all_inserts.append(seq_1hot)
                    # print(seq_1hot[655241:655241+19] == seq_1hot[655260:655260+19])
            all_inserts = np.array(all_inserts)
        
        elif orientation == "convergent":
            for background_seq in background_seqs:
                for i in seq_coords_df.index:
                    seq_1hot = background_seq.copy()
                    this_row = seq_coords_df.iloc[i]
                    seq_1hot_CTCF_pad_left = dna_io.dna_1hot(genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding).upper())
                    if this_row.strand == '-': 
                        seq_1hot_CTCF_pad_left = dna_io.hot1_rc(seq_1hot_CTCF_pad_left)
                    # print(this_row.strand, Seq(genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding).upper()))
                    seq_1hot_CTCF_pad_right = dna_io.hot1_rc(seq_1hot_CTCF_pad_left)
                    for o in range(len(offsets)):
                        if o % 2 == 0:
                            seq_1hot[offsets[o] : offsets[o] + len(seq_1hot_CTCF_pad_left)] = seq_1hot_CTCF_pad_left
                        else:
                            seq_1hot[offsets[o] : offsets[o] + len(seq_1hot_CTCF_pad_right)] = seq_1hot_CTCF_pad_right
                    all_inserts.append(seq_1hot)
                    # print(seq_1hot[655241:655241+19] == seq_1hot[655260:655260+19])
            all_inserts = np.array(all_inserts) 
        
        elif orientation == "divergent":
            for background_seq in background_seqs:
                for i in seq_coords_df.index:
                    seq_1hot = background_seq.copy()
                    this_row = seq_coords_df.iloc[i]
                    seq_1hot_CTCF_pad_left = dna_io.dna_1hot(genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding).upper())
                    if this_row.strand == '-': 
                        seq_1hot_CTCF_pad_left = dna_io.hot1_rc(seq_1hot_CTCF_pad_left)
                    # print(this_row.strand, Seq(genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding).upper()))
                    seq_1hot_CTCF_pad_right = dna_io.hot1_rc(seq_1hot_CTCF_pad_left)
                    for o in range(len(offsets)):
                        if o % 2 == 0:
                            seq_1hot[offsets[o] : offsets[o] + len(seq_1hot_CTCF_pad_left)] = seq_1hot_CTCF_pad_left
                        else:
                            seq_1hot[offsets[o] : offsets[o] + len(seq_1hot_CTCF_pad_right)] = seq_1hot_CTCF_pad_right
                    all_inserts.append(seq_1hot)
                    # print(seq_1hot[655241:655241+19] == seq_1hot[655260:655260+19])
            all_inserts = np.array(all_inserts) 
        
        print(f"padding = {padding} || Prediction")
        pred = seqnn_model.predict(all_inserts, batch_size=10)

        scd_score = []

        # plt.figure(figsize=(8*3,5*3))
        
#         if plotting == True:
            
#             plt.figure(figsize=(8*3,5*3))
            
#             print(f"padding = {padding} || Plotting")
            
#             target_ind = 0
#             vlim = .5
#             bin_mid = target_map_size//2
#             window = 50
#             for i in range(40):
#                 insert_pred = pred[i]

#                 plt.subplot(8,5, i+1)
#                 im = plt.matshow(
#                         akita_utils.from_upper_triu(  
#                         insert_pred[:,target_ind], target_map_size,hic_diags),
#                         vmin=-1*vlim, vmax=vlim, fignum=False,cmap='RdBu_r')
#                 plt.colorbar(im, fraction=0.046, pad=0.04)
#                 plt.title('insert-scd: '+str(  np.sqrt( (insert_pred**2).sum(axis=0)  ).mean() ) 
#                           ) 
#                 #plt.axis([ bin_mid  - window,bin_mid+window,bin_mid-window, bin_mid+window])
#             plt.tight_layout()
#             # plt.savefig(f"Padding_{padding}_{orientation}.png")
#             plt.show()
        
        print(f"padding = {padding} || SCD Calculation")
        for i in range(nr_seq * nr_bg):
            insert_pred = pred[i]
            # print(i, np.sqrt( (insert_pred**2).sum(axis=0)))
            scd_score.append(np.sqrt( (insert_pred**2).sum(axis=0)  ).mean())
        
        np_scd_score = np.array(scd_score)
        reshaped_scd_score = np_scd_score.reshape((nr_bg, nr_seq))
        # seq_coords_df[f"Pad_{padding}_SCD"] = scd_score
        
        for bs in range(len(background_seqs)):
            seq_coords_df[f"Pad_{padding}_b{bs}_SCD"] = reshaped_scd_score[bs, :]
        
    return seq_coords_df


In [14]:
pad_list = [k for k in range(50, 60)]

In [15]:
multiple_padding(seq_coords_df, background_seqs, pad_list, orientation = "divergent", plotting=False, 
                     num_inserts=2, 
                     one_side_radius=100, 
                     motif_len=19)

padding = 50 || Started working on padding = 50
padding = 50 || Prediction
40/40 [==============================] - 452s 11s/step
padding = 50 || SCD Calculation
padding = 51 || Started working on padding = 51
padding = 51 || Prediction
40/40 [==============================] - 450s 11s/step
padding = 51 || SCD Calculation
padding = 52 || Started working on padding = 52
padding = 52 || Prediction
40/40 [==============================] - 449s 11s/step
padding = 52 || SCD Calculation
padding = 53 || Started working on padding = 53
padding = 53 || Prediction
40/40 [==============================] - 450s 11s/step
padding = 53 || SCD Calculation
padding = 54 || Started working on padding = 54
padding = 54 || Prediction
40/40 [==============================] - 449s 11s/step
padding = 54 || SCD Calculation
padding = 55 || Started working on padding = 55
padding = 55 || Prediction
40/40 [==============================] - 449s 11s/step
padding = 55 || SCD Calculation
padding = 56 || Started work

/tmp/SLURM_6752212/ipykernel_207953/2672018541.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  seq_coords_df[f"Pad_{padding}_b{bs}_SCD"] = reshaped_scd_score[bs, :]
/tmp/SLURM_6752212/ipykernel_207953/2672018541.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  seq_coords_df[f"Pad_{padding}_b{bs}_SCD"] = reshaped_scd_score[bs, :]
/tmp/SLURM_6752212/ipykernel_207953/2672018541.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

,index,chrom,start,end,strand,genomic_SCD,Pad_50_b0_SCD,Pad_50_b1_SCD,Pad_50_b2_SCD,Pad_50_b3_SCD,...,Pad_59_b0_SCD,Pad_59_b1_SCD,Pad_59_b2_SCD,Pad_59_b3_SCD,Pad_59_b4_SCD,Pad_59_b5_SCD,Pad_59_b6_SCD,Pad_59_b7_SCD,Pad_59_b8_SCD,Pad_59_b9_SCD
0,0,chr12,35192359,35192378,-,53.656250,35.093750,34.843750,32.875000,35.093750,...,35.531250,34.843750,32.812500,35.093750,40.562500,43.125000,34.875000,46.812500,33.750000,34.843750
1,1,chr8,102781112,102781131,-,53.500000,28.875000,27.156250,27.765625,29.578125,...,28.875000,27.781250,28.875000,30.453125,32.062500,37.062500,29.453125,38.218750,32.312500,28.875000
2,2,chr5,49961991,49962010,-,53.312500,30.656250,28.390625,28.968750,32.781250,...,31.953125,29.000000,29.125000,31.234375,33.125000,37.062500,30.734375,39.343750,33.187500,28.937500
3,3,chr9,26776356,26776375,-,53.125000,30.140625,28.875000,29.093750,33.218750,...,32.000000,30.343750,29.546875,34.000000,35.031250,37.062500,30.843750,41.968750,33.750000,29.718750
4,4,chr9,51152589,51152608,-,53.062500,28.875000,26.437500,26.937500,28.953125,...,28.875000,26.609375,28.234375,29.296875,30.765625,37.062500,28.906250,37.187500,27.921875,28.875000
5,5,chr8,58566264,58566283,+,52.875000,23.968750,25.921875,22.187500,27.125000,...,26.531250,26.203125,24.875000,27.781250,28.875000,33.906250,28.875000,35.500000,25.812500,26.640625
6,6,chr12,40837731,40837750,+,52.843750,28.953125,27.062500,28.875000,32.500000,...,29.078125,28.125000,28.875000,33.062500,30.843750,37.062500,30.296875,41.468750,32.656250,28.921875
7,7,chr8,36523116,36523135,-,52.500000,28.875000,26.484375,28.750000,30.562500,...,28.875000,26.671875,28.875000,30.609375,30.671875,37.062500,29.500000,38.562500,31.718750,28.875000
8,8,chr6,101261721,101261740,-,52.500000,28.875000,28.078125,27.843750,30.218750,...,28.875000,28.656250,28.890625,30.125000,32.250000,37.062500,29.187500,37.812500,31.296875,28.875000
9,9,chr11,57467954,57467973,-,52.375000,30.406250,29.937500,30.500000,32.687500,...,30.578125,30.437500,30.546875,33.000000,35.218750,37.062500,32.218750,43.937500,33.781250,30.546875


In [16]:
seq_coords_df.to_csv("./50-60pads_divergent.csv", index=False)

In [17]:
# offsets = []

# print(seq_length//2)
# print(seq_length//2 - multi_insert_length//2)
# for i in range(num_inserts):
#     offsets.append( seq_length//2 - multi_insert_length//2 + i * (multi_insert_length//2) + spacer)
# print(offsets)

In [18]:
# all_inserts = []

# # inserting only to the first background sequence
# for background_seq in background_seqs:
#     for i in seq_coords_df.index:
#         seq_1hot = background_seq.copy()
#         this_row = seq_coords_df.iloc[i]
#         seq_1hot_CTCF_pad = dna_io.dna_1hot(genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding).upper())
#         if this_row.strand == '-': 
#             seq_1hot_CTCF_pad = dna_io.hot1_rc(seq_1hot_CTCF_pad)
#         # print(this_row.strand, Seq(genome_open.fetch(this_row.chrom, this_row.start-padding, this_row.end+padding).upper()))
#         for offset in offsets:
#             seq_1hot[offset : offset + len(seq_1hot_CTCF_pad)] = seq_1hot_CTCF_pad
#         all_inserts.append(seq_1hot)
#         # print(seq_1hot[655241:655241+19] == seq_1hot[655260:655260+19])
# all_inserts = np.array(all_inserts)

In [19]:
# all_inserts.shape

In [20]:
# pred = seqnn_model.predict(all_inserts, batch_size=10)

In [21]:
# pred.shape

In [22]:
# scd_score = []

In [23]:
# plt.figure(figsize=(8*3,5*3))

# target_ind = 0
# vlim = .5
# bin_mid = target_map_size//2
# window = 50
# for i in range(40):
#     insert_pred = pred[i]
#     print(i, np.sqrt( (insert_pred**2).sum(axis=0)))
#     scd_score.append(np.sqrt( (insert_pred**2).sum(axis=0)  ).mean())
    
#     plt.subplot(8,5, i+1)
#     im = plt.matshow(
#             akita_utils.from_upper_triu(  
#             insert_pred[:,target_ind], target_map_size,hic_diags),
#             vmin=-1*vlim, vmax=vlim, fignum=False,cmap='RdBu_r')
#     plt.colorbar(im, fraction=0.046, pad=0.04)
#     plt.title('insert-scd: '+str(  np.sqrt( (insert_pred**2).sum(axis=0)  ).mean() ) 
#               ) 
#     #plt.axis([ bin_mid  - window,bin_mid+window,bin_mid-window, bin_mid+window])
# plt.tight_layout()
# plt.show()

In [24]:
# for i in range(400):
#     insert_pred = pred[i]
#     # print(i, np.sqrt( (insert_pred**2).sum(axis=0)))
#     scd_score.append(np.sqrt( (insert_pred**2).sum(axis=0)  ).mean())

In [25]:
# len(scd_score)

In [26]:
# np_scd_score = np.array(scd_score)

In [27]:
# np_scd_score[2,]

In [28]:
# reshaped_np_scd_score = np_scd_score.reshape((10,40))

In [29]:
# reshaped_np_scd_score[:,31]

In [30]:
# reshaped_np_scd_score.mean(axis=0)

In [31]:
# seq_coords_df[f"Pad{padding}_SCD"] = scd_score

In [32]:
# seq_coords_df